RECORDAR DESZIPEAR PROPERATTI.ZIP

In [2]:
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd
import numpy as np

In [21]:
data = pd.read_csv("../properatti.csv", index_col=0)
data.shape

(121220, 25)

In [40]:
data.dtypes

operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: object

In [41]:
data.rooms.value_counts()

3.0     12567
2.0     12433
4.0      7937
1.0      7377
5.0      3782
6.0      1403
7.0       770
8.0       409
10.0      226
9.0       184
11.0       72
12.0       65
13.0       33
14.0       27
15.0       26
17.0       16
16.0       11
20.0       11
22.0        8
18.0        6
21.0        5
30.0        4
25.0        4
19.0        4
32.0        3
23.0        2
24.0        1
27.0        1
29.0        1
31.0        1
28.0        1
Name: rooms, dtype: int64

In [42]:
mask_residencial= data.property_type != 'store'
tipos_residenciales=data.property_type[mask_residencial]
rooms_residenciales=data.rooms[mask_residencial]
df_residencial=pd.DataFrame(tipos_residenciales)
df_residencial['rooms']=rooms_residenciales

##### Intentamos ver si los datos faltantes de rooms disminuyen si descontamos las tiendas

In [43]:
shape_df=df_residencial.shape
count_res=shape_df[0]


In [44]:
nulos_room=df_residencial.rooms.isna().sum(axis=0)
pctje_nulos_rooms=nulos_room/count_res
pctje_nulos_rooms

0.5961446482867001

Veamos cuantas rooms tienen las tiendas

In [45]:
mask_comercial= data.property_type == 'store'
tipos_comerciales=data.property_type[mask_comercial]
rooms_comerciales=data.rooms[mask_comercial]
df_comercial=pd.DataFrame(tipos_comerciales)
df_comercial['rooms']=rooms_comerciales
df_comercial.rooms.value_counts()

1.0     55
2.0     17
3.0     13
7.0      6
5.0      4
4.0      4
10.0     3
6.0      2
8.0      1
Name: rooms, dtype: int64

#### Recordemos que el porcentaje original de nulos era similar, 

Veamos si los faltantes de expensas disminuyen sacando tiendas y casas

In [46]:
mascara_sin_casas=df_residencial.property_type !='house'
expensas_residenciales=data.expenses[mask_residencial]
df_residencial['expenses']=expensas_residenciales
expensas_deps_ph=expensas_residenciales[mascara_sin_casas]
null_exp_clean=expensas_deps_ph.isnull().sum()
shape_exp_clean=expensas_deps_ph.shape
pctje_null_exp_clean=null_exp_clean/shape_exp_clean[0]
pctje_null_exp_clean
type_deps_ph=df_residencial.property_type[mascara_sin_casas]
type_deps_ph.value_counts()

apartment    71065
PH            5751
Name: property_type, dtype: int64

In [47]:
expensas_deps_ph.isnull().sum()/expensas_deps_ph.shape[0]

0.8496146636117475

Probemos ahora sacando los nulos de rooms para tratar de aplicar un promedio del tamaño de las rooms

In [48]:
mask_rooms_no_nulas=data.rooms.notnull()
rooms_no_nulas=data.rooms.loc[mask_rooms_no_nulas]
sf_room_no_nulas=data.surface_covered_in_m2.loc[mask_rooms_no_nulas]
df_rooms_clean=pd.DataFrame(rooms_no_nulas)
df_rooms_clean['m2_covered']=sf_room_no_nulas
mask_rooms_no_nulas.value_counts()

False    73830
True     47390
Name: rooms, dtype: int64

In [49]:
m2_nulos_rooms_clean=df_rooms_clean.m2_covered.isnull().sum()
shape_rooms_clean=df_rooms_clean.shape
pctje_nulos_rooms_clean=m2_nulos_rooms_clean/shape_rooms_clean[0]
pctje_nulos_rooms_clean

0.12789618062882466

El pctje de m2 cubiertos es ligeramente menor, por lo que no hay mucha correlacion entre nulidad de rooms y nulidad de m2_covered
Hay que inputar los m2 cubiertos faltantes antes de poder avanzar a tener un promedio de cuantos m2 tiene cada habitación
Para imputar las rooms, me imagino hay que inputar promedio por tipo de propiedad, hace falta por barrio, o alguna caracteristica extra?? Para mi no hace falta

### Veamos el tema de los m2 faltantes

Primera cosa a tener en cuenta, los m2 covered tienen un pctje nulo del 16% y los totales tienen un pctje del 32%
Price_per_m2 tiene un faltante del 28%
Price usd per m2 tiene un faltante del 43%
Todos las variantes de precios totales tienen un faltante aprox del 17%

# Verificar si existen filas con todos los datos faltantes

#Nota: revisar estos datos despues de haber completado todas las columnas con los datos extraidos del campo descripcion

In [ ]:
mask_nan_sup_total=data["surface_total_in_m2"].isna()
mask_nan_sup_cubierta=data["surface_covered_in_m2"].isna()
mask_nan_rooms=data["rooms"].isna()

In [ ]:
mask_nan_columnas = (mask_nan_sup_total == True) & (mask_nan_sup_cubierta == True) &(mask_nan_rooms == True) &(mask_nan_uds_m2 == True) 
mask_nan_columnas.value_counts()

# Se desea recuperar informacion faltante en columnas: Superficie, precio, rooms, floor.

## Columna Superficie

In [5]:
import re

In [6]:
serie_descripcion = data["description"]

In [ ]:
pattern_sup_m2= "(?P<sup>\d+.?,?\d+?.?m2|\d+.?,?\d+?.?M2|\d+.?,?\d+?.?METROS|\d+.?,?\d+?.?Metros|\d+.?,?\d+?.?metros|\d+.?,?\d+?.?mts|\d+.?,?\d+?.?Mts|\d+.?,?\d+?.?MTS)"

In [ ]:
pattern_sup_m2_regex=re.compile(pattern_sup_m2)

In [ ]:
import numpy as np
sup_match = serie_descripcion.apply(lambda x: x if x is np.NaN else pattern_sup_m2_regex.search(x))


In [ ]:
sup_match.sample(10)

In [ ]:
mask_sup_match_notnull= sup_match.notnull()

data.loc[mask_sup_match_notnull, "Sup_m2_Clean"] = sup_match[mask_sup_match_notnull].apply(lambda x: x.group(0))


In [ ]:
data.loc[mask_sup_match_notnull, ["surface_total_in_m2", "surface_covered_in_m2", "Sup_m2_Clean"]]

In [ ]:
serie_sup_m2_clean=data["Sup_m2_Clean"]

In [ ]:
pattern_sup_m2_dos="(?P<supdos>\d+.?,?\d+?)"
pattern_sup_m2_regex_dos=re.compile(pattern_sup_m2_dos)
sup_match_dos = serie_sup_m2_clean.apply(lambda x: x if x is np.NaN else pattern_sup_m2_regex_dos.search(x))
sup_match_dos.sample(10)

In [ ]:
mask_sup_match_notnull_dos= sup_match_dos.notnull()

data.loc[mask_sup_match_notnull_dos, "Sup_m2_Clean_dos"] = sup_match_dos[mask_sup_match_notnull_dos].apply(lambda x: x.group(0))

In [ ]:
data.loc[mask_sup_match_notnull, ["surface_total_in_m2", "surface_covered_in_m2", "Sup_m2_Clean_dos"]]

# Se completan datos faltantes de sup cubierta con sup total

## ARREGLAR - ES AL REVES. USAREMOS SUP TOTAL

In [17]:
data_limpia= data

In [ ]:
data_limpia["surface_covered_in_m2"].fillna(data_limpia["surface_total_in_m2"],inplace=True)
data_limpia

In [ ]:
data_limpia["surface_covered_in_m2"].fillna(data_limpia["Sup_m2_Clean_dos"],inplace=True)
data_limpia

% de info recuperada = 45%

In [ ]:
data_limpia["surface_covered_in_m2"].isnull().sum()/data["surface_covered_in_m2"].isnull().sum()*100

In [ ]:
data_limpia["surface_covered_in_m2"].sample(20)

#eliminar datos erroneos q quedaron de regex!!!

In [ ]:
mascara_83 = data_limpia['surface_covered_in_m2'] == "83M2"
data_limpia[mascara_83]

In [ ]:
mascara_coma = data_limpia['surface_covered_in_m2'] == '3,5'
data_limpia[mascara_coma]

In [ ]:
data_replace = data_limpia['surface_covered_in_m2'].replace(",", "." )
data_replace[mascara_coma]

# se agrupa por tipo de propiedad para sacar la mediana de los m2 cubiertos y completar los datos faltantes en esa columna

In [ ]:
data_limpia.groupby('property_type')['surface_covered_in_m2'].median()

grafico para ver sup cubierta por tipo de propiedad - arreglar error primero y verificar grafico

In [ ]:
import seaborn as sns
sns.scatterplot(data= data_limpia, y="surface_covered_in_m2", x="property_type")

# Columna Rooms

In [7]:
pattern_rooms= "(?P<dorm>\d+.?Dorm|\d+.?dorm|\d+.?DORM|\d+.?AMB|\d+.?Amb|\d+.?amb|\d+.?hab|\d+.?HAB|\d+.?Hab|monoamb)"

In [8]:
pattern_rooms_regex=re.compile(pattern_rooms)

In [9]:
room_match = serie_descripcion.apply(lambda x: x if x is np.NaN else pattern_rooms_regex.search(x))

In [12]:
room_match.sample(10)

111121                                                 None
75171                                                  None
43445     <re.Match object; span=(95, 101), match='3 DORM'>
83037                                                  None
29125       <re.Match object; span=(25, 30), match='3 amb'>
108051                                                 None
72870     <re.Match object; span=(194, 199), match='2 amb'>
56895                                                  None
39960       <re.Match object; span=(22, 27), match='2 amb'>
68001       <re.Match object; span=(16, 21), match='2 AMB'>
Name: description, dtype: object

se saca la parte entera del str

In [13]:
mask_rooms_notnull= room_match.notnull()

data.loc[mask_rooms_notnull, "rooms_clean"] = room_match[mask_rooms_notnull].apply(lambda x: x.group(0))


In [15]:
serie_rooms_clean=data["rooms_clean"]
serie_rooms_clean.sample(5)


53032        NaN
45343     3 dorm
91643        NaN
120552       NaN
28617      3 amb
Name: rooms_clean, dtype: object

In [16]:
pattern_rooms_dos="(?P<dorm_dos>\d+.?)"
pattern_rooms_regex_dos=re.compile(pattern_rooms_dos)
room_match_dos = serie_rooms_clean.apply(lambda x: x if x is np.NaN else pattern_rooms_regex_dos.search(x))
room_match_dos.sample(10)

32535                                            NaN
81964     <re.Match object; span=(0, 2), match='1 '>
101906    <re.Match object; span=(0, 2), match='2 '>
97791                                            NaN
61958     <re.Match object; span=(0, 2), match='4 '>
116008    <re.Match object; span=(0, 2), match='1 '>
22445                                            NaN
62508     <re.Match object; span=(0, 2), match='1 '>
97961                                            NaN
32629     <re.Match object; span=(0, 2), match='4 '>
Name: rooms_clean, dtype: object

In [18]:
mask_room_match_notnull_dos= room_match_dos.notnull()

data.loc[mask_room_match_notnull_dos, "rooms_clean_dos"] = room_match_dos[mask_room_match_notnull_dos].apply(lambda x: x.group(0))

In [20]:
data_limpia["rooms"].fillna(data_limpia["rooms_clean_dos"],inplace=True)
data_limpia

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,rooms_clean,rooms_clean_dos
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,1550.000000,NaN,2,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,2 AMB,2
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,NaN,NaN
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,1309.090909,NaN,2,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,2 AMB,2
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,NaN,3,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,3 amb,3
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,1828.571429,NaN,1,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,1 DORM,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,NaN,...,9354.838710,NaN,2,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...,2 dorm,2
121216,sell,house,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436080.0,NaN,NaN,NaN,...,1383.333333,NaN,3,NaN,http://www.properati.com.ar/1cja6_venta_casa_b...,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...,https://thumbs4.properati.com/2/PCc3WuQDjpNZc4...,3 Dorm,3
121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",-34.570639,-58.475596,...,3371.794872,NaN,NaN,NaN,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...,monoamb,NaN
121218,sell,apartment,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,NaN,NaN,NaN,NaN,...,1997.916667,NaN,2,NaN,http://www.properati.com.ar/1cja8_venta_depart...,"2 Amb al contrafrente, luminoso. El departame...",2 amb. C/ dep. de servicio al contrafrente| Re...,https://thumbs4.properati.com/8/Q12PTvU6BQJ0ib...,2 Amb,2


% de info recuperada = 46%

In [22]:
data_limpia["rooms"].isnull().sum()/data["rooms"].isnull().sum()*100

46.27116348367872

# se pueden recuperar datos de la columan precios ARS_m2?

In [ ]:
df_ars_m2 = data['price_per_m2']
df_ars_m2.head(10)

In [ ]:
df_usd_m2 = data['price_usd_per_m2']
df_usd_m2.head(10)

In [ ]:
mask_nan_uds_m2= df_usd_m2.isna()
mask_nan_uds_m2.head(10)

In [ ]:
mask_nan_ars_m2= df_ars_m2.isna()
mask_nan_ars_m2.head(10)

In [ ]:
mask = (mask_nan_ars_m2 == False) & (mask_nan_uds_m2== True)
mask.value_counts()

Se pueden recuperar 24308 valores de la columna precios argentinos.

#NOTA: EVALUAR ELIMINAR LOS VALORES Q NO SON PESOS ARGENTINOS PARA PODER HACER UNA CONVERSION DE TODOS LOS VALORES A USD

Evaluar conversion entre ARS y USD

In [ ]:
import numpy as np
change = data['price_per_m2'] / data['price_usd_per_m2']
change.head(10)

In [ ]:
data.groupby('property_type')['price_usd_per_m2'].median()